In [ ]:
import pandas as pd
from anyio import sleep
import numpy as np
from datetime import time
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

In [ ]:
def load_and_aggregate_routes_with_adress():
    locations = pd.read_excel('data/locations_with_coords.xlsx', header=0)
    routes = pd.read_excel('data/StrategischerTourenplan_HSLU.xlsx', sheet_name=0, header=0)
    routes_grouped = routes.groupby(['Tournummer','VST']).first().reset_index().__deepcopy__()
    locations.rename(columns={'VST-Nummer':'SAP'}, inplace=True)
    duplicate_locations = locations[locations.duplicated(subset='SAP', keep=False)]
    duplicate_locations = locations[locations.duplicated(subset='SAP', keep=False)]
    routes_with_address = routes_grouped.join(locations.set_index('SAP'), on='SAP', how='left')
    return routes_with_address, routes, locations

In [ ]:
routes_with_address, routes, locations = load_and_aggregate_routes_with_adress()

In [ ]:
routes_with_address.columns

In [ ]:
routes_with_address.head()

In [ ]:
routes_with_address.columns

In [ ]:
from datetime import time

In [ ]:
def convert_time_column_to_datetime(x):
    return pd.to_datetime(x.apply(lambda x: x.isoformat()))

In [ ]:
def get_timestamp_bins_from_a_time_column(x, freq="30min"):
    bins = pd.timedelta_range("00:00:00", "23:30:00", freq=freq)
    bins_datetime = pd.to_datetime(bins.astype(np.int64)).time
    bins_real_datetime = [pd.to_datetime(bin.isoformat()) for bin in bins_datetime]
    return bins_real_datetime

In [ ]:
def get_bin_cuts_from_a_time_column(x, freq="30min"):
    datetime_columns = convert_time_column_to_datetime(x)
    bins_real_datetime = get_timestamp_bins_from_a_time_column(x, freq=freq)
    bin_cuts = pd.cut(datetime_columns, bins=bins_real_datetime)
    return bin_cuts

In [ ]:
def add_time_bins_from_column_to_plot(col, ax, freq="30min"):
    bin_cuts =  get_bin_cuts_from_a_time_column(col, freq=freq)
    col.groupby(bin_cuts).count().plot(kind="bar", ax=ax)
    labels = [(l.get_text().split(' ')[1]).replace(']','').replace(',','') for l in ax.get_xticklabels()]
    _ = ax.set_xticklabels(labels)
    ax.set_xlabel('Tageszeit (30min-Intervalle)')

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))  #create figure and axes
add_time_bins_from_column_to_plot(routes_with_address['Abfahrtszeit Beladestelle'], ax=ax, freq="30min")
#add_time_bins_from_column_to_plot(routes_with_address['Tourendezeit'], ax=ax, freq="30min")

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))  #create figure and axes
#add_time_bins_from_column_to_plot(routes_with_address['Abfahrtszeit Beladestelle'], ax=ax, freq="30min")
add_time_bins_from_column_to_plot(routes_with_address['Tourendezeit'], ax=ax, freq="30min")

In [ ]:
import seaborn as sns

In [ ]:
def get_bin_counts(col, freq="30min"):
    bin_cuts =  get_bin_cuts_from_a_time_column(col, freq=freq)
    data = pd.DataFrame(col.groupby(bin_cuts).count())
    return data.values

In [ ]:
bins_real_datetime = get_timestamp_bins_from_a_time_column(routes_with_address['Tourendezeit'], freq="30min")
#bins_real_datetime

In [ ]:
bins_real_datetime = get_timestamp_bins_from_a_time_column(routes_with_address['Tourendezeit'], freq="30min")

In [ ]:
col = routes_with_address['Tourendezeit']
bin_cuts =  get_bin_cuts_from_a_time_column(col, freq="30min")
data = pd.DataFrame(col.groupby(bin_cuts).count())
#data = data.rename(columns={'Tourendezeit':'Counts'}).reset_index()
data.index.name

In [ ]:
def add_hist_to_plot(col, ax, freq="30min"):
    bin_cuts =  get_bin_cuts_from_a_time_column(col, freq)
    data = pd.DataFrame(col.groupby(bin_cuts).count())
    name = data.index.name
    data = data.rename(columns={name:'Counts'}).reset_index()
    return sns.barplot(data, x=name, y='Counts', ax=ax, alpha=0.8)

In [ ]:
fig, ax = plt.subplots(figsize=(15,5)) 
add_hist_to_plot(routes_with_address['Abfahrtszeit Beladestelle'], ax, freq="15min")
add_hist_to_plot(routes_with_address['Tourendezeit'], ax, freq="15min")
labels = [(l.get_text().split(' ')[1]).replace(']','').replace(',','') for l in ax.get_xticklabels()]
_ = ax.set_xticklabels(labels, rotation=270)
ax.legend(labels=['Abfahrtszeit Beladestelle','Tourendezeit'], loc='best', fontsize=12)
ax.set_xlabel('Day Time Intervals (15min)', fontsize=12)
ax.set_ylabel('Number of Vehicles', fontsize=12)
eight = mlines.Line2D([], [], color=plt.cm.tab10.colors[0], marker='s', ls='', label='Departure Time Loading Station')
nine = mlines.Line2D([], [], color=plt.cm.tab10.colors[1], marker='s', ls='', label='End of Tour')
start, end = ax.get_ylim()
ax.yaxis.set_ticks(np.arange(start, end, 50))
plt.legend(handles=[eight, nine], fontsize=12)
plt.tight_layout()

In [ ]:
routes_with_address_monday = routes_with_address[routes_with_address['Wochentag']=='Montag']

In [ ]:
fig, axes = plt.subplots(6,1, figsize=(15,10), sharex=True, sharey=True) 

ax = axes[0]
routes_with_address_monday = routes_with_address[routes_with_address['Wochentag']=='Montag']
add_hist_to_plot(routes_with_address_monday['Abfahrtszeit Beladestelle'], ax, freq="15min")
add_hist_to_plot(routes_with_address_monday['Tourendezeit'], ax, freq="15min")

ax.legend(labels=['Abfahrtszeit Beladestelle','Tourendezeit'], loc='best', fontsize=12)
ax.set_xlabel('Day Time Intervals (15min)', fontsize=12)
ax.set_ylabel('Vehicles', fontsize=12)
eight = mlines.Line2D([], [], color=plt.cm.tab10.colors[0], marker='s', ls='', label='Departure Time Loading Station')
nine = mlines.Line2D([], [], color=plt.cm.tab10.colors[1], marker='s', ls='', label='End of Tour')
axes[0].set_title('Monday')

start, end = ax.get_ylim()
for axi in axes:
    axi.yaxis.set_ticks(np.arange(start, end, 10))
ax.legend(handles=[eight, nine], fontsize=12)

routes_with_address_monday = routes_with_address[routes_with_address['Wochentag']=='Dienstag']
add_hist_to_plot(routes_with_address_monday['Abfahrtszeit Beladestelle'], axes[1], freq="15min")
add_hist_to_plot(routes_with_address_monday['Tourendezeit'], axes[1], freq="15min")
axes[1].yaxis.set_ticks(np.arange(start, end, 10))
axes[1].set_title('Tuesday')
axes[1].set_ylabel('Vehicles', fontsize=12)


routes_with_address_monday = routes_with_address[routes_with_address['Wochentag']=='Mittwoch']
add_hist_to_plot(routes_with_address_monday['Abfahrtszeit Beladestelle'], axes[2], freq="15min")
add_hist_to_plot(routes_with_address_monday['Tourendezeit'], axes[2], freq="15min")
axes[2].yaxis.set_ticks(np.arange(start, end, 10))
axes[2].set_title('Wednesday')
axes[2].set_ylabel('Vehicles', fontsize=12)

routes_with_address_monday = routes_with_address[routes_with_address['Wochentag']=='Donnerstag']
add_hist_to_plot(routes_with_address_monday['Abfahrtszeit Beladestelle'], axes[3], freq="15min")
add_hist_to_plot(routes_with_address_monday['Tourendezeit'], axes[3], freq="15min")
axes[3].yaxis.set_ticks(np.arange(start, end, 10))
axes[3].set_title('Thursday')
axes[3].set_ylabel('Vehicles', fontsize=12)

routes_with_address_monday = routes_with_address[routes_with_address['Wochentag']=='Freitag']
add_hist_to_plot(routes_with_address_monday['Abfahrtszeit Beladestelle'], axes[4], freq="15min")
add_hist_to_plot(routes_with_address_monday['Tourendezeit'], axes[4], freq="15min")
axes[4].yaxis.set_ticks(np.arange(start, end, 10))
axes[4].set_title('Friday')
axes[4].set_ylabel('Vehicles', fontsize=12)

routes_with_address_monday = routes_with_address[routes_with_address['Wochentag']=='Samstag']
add_hist_to_plot(routes_with_address_monday['Abfahrtszeit Beladestelle'], axes[5], freq="15min")
add_hist_to_plot(routes_with_address_monday['Tourendezeit'], axes[5], freq="15min")
axes[5].yaxis.set_ticks(np.arange(start, end, 10))
axes[5].set_title('Saturday')
axes[5].set_ylabel('Vehicles', fontsize=12)

axes[5].set_xlabel('Day Time (15min Intervals)')


labels = [(l.get_text().split(' ')[1]).replace(']','').replace(',','') for l in axes[5].get_xticklabels()]
_ = axes[5].set_xticklabels(labels, rotation=270)
plt.tight_layout()